In [1]:
import model as Model
import torch

In [2]:
with open('../Textbook-text-data.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

In [3]:
len(text)

3180566

In [4]:
chars = sorted(list(set(text)))
''.join(chars)

'\n !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_abcdefghijklmnopqrstuvwxyz{|}¬´·×âîŁžˆΔαβλπτ–—‘’“”•…′→⇐∈∑−∕∖∗√≈≠≤≥≶⋅◦⟨⟩𝖺𝖼𝗆𝗉𝗋𝗍𝗒𝚗'

In [5]:
vocab_size = len(chars)
vocab_size

144

In [6]:
stoi = {ch : i for i, ch in enumerate(chars)}
itos = {i : ch for i, ch in enumerate(chars)}
encode = lambda text : [stoi[char] for char in text]
decode = lambda encoded_text : ''.join([itos[item] for item in encoded_text])

In [7]:
data = torch.tensor(encode(text), dtype = torch.long)

In [8]:
# Train and test split
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]
train_data.shape, val_data.shape

(torch.Size([2862509]), torch.Size([318057]))

In [9]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    x = x.to(device)
    y = y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval();
    
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for i in range(eval_iters):
            X, Y = get_batch(split)
            X = X.to(device)
            Y = Y.to(device)
            logits, loss = model(X, Y)
            losses[i] = loss.item()
        out[split] = losses.mean()
        
    model.train()
    return out

In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import random
import re

# Hyperparameters
batch_size = 64   # B
block_size = 256  # T (Context length)
n_layers = 6      # Number of blocks or units of the decoder in the architecture
n_embd = 384      # num_heads * head_size
num_heads = 6     # Number of attention heads in multiheaded attention
head_size = n_embd // num_heads  # Sequence length processed by a single head of attention
max_iters = 5000
eval_iters = 200
eval_interval = 500
learning_rate = 3e-4
dropout = 0.2      # % dropout
device = 'cuda' if torch.cuda.is_available() else 'cpu'




# One head of self attention in multiheaded attentino
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()  # Initialize parameters for the derived class object
        
        self.head_size = head_size
        self.query = nn.Linear(n_embd, head_size, bias = False) # Part of sequence being processes currently
        self.key = nn.Linear(n_embd, head_size, bias = False)   # Parts of the sequence to attend to
        self.value = nn.Linear(n_embd, head_size, bias = False) # Parts of sequence other than the current part
        # tril will have no learnable parameters
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) 
        
        self.dropout = nn.Dropout(dropout)
        
    # Forward method of a derived class of nn.Module is called by the __call__ method of the base nn.Module
    # class. Objects of classes with __call__ method are called 'callable objects'
    def forward(self, x):
        B, T, C = x.shape # B - batch_size, T - block_size (time dimension), C - channels
        k = self.key(x) # (B, T, head_size)
        q = self.key(x) # (B, T, h_s)
        wei = (q @ k.transpose(-2, -1)) * (self.head_size**(-0.5)) # (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim = -1)
        wei = self.dropout(wei)
        
        # Weighted aggregation of values
        v = self.value(x)  # (B, T, h_s)
        out = wei @ v      # (B, T, h_s)
        return out;
        
        
# Multiple heads of attention
class MultiHeadedAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim = -1)
        out = self.dropout(self.proj(out))
        return out
        
        
# Feed Forward Neural Network
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
        
    def forward(self, x):
        out = self.network(x)
        return out
    

# A block in the transformer decoder
class Block(nn.Module):
    def __init__(self, n_embd, num_heads):
        super().__init__()
        self.self_attention = MultiHeadedAttention(num_heads, head_size)
        self.feed_forward_network = FeedForward(n_embd)
        self.layer_norm_1 = nn.LayerNorm(n_embd)
        self.layer_norm_2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        # Adding x -> The purpose of the residual connection is to ensure that important
        # information from the input sequence is preserved and propagated through the network.
        # Also for improved gradient flow (no vanishing or exploding gradients)
        # Original paper => Self Attention -> Add and Layer_Norm -> Feed_Forward
        # More recently =>  Layer_Norm -> Self_Attention -> Add and Feed_Forward
        x = x + self.self_attention(self.layer_norm_1(x)) 
        x = x + self.feed_forward_network(self.layer_norm_2(x))
        return x
        
        
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # https://towardsdatascience.com/master-positional-encoding-part-i-63c05d90a0c3
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # Positional encoding
        self.blocks = nn.Sequential(*[Block(n_embd, num_heads) for _ in range(n_layers)])
        self.final_layer_norm = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size) # lanuage modelling head
        
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
                
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
            
    
    def forward(self, idx, targets = None):
        B, T = idx.shape
        
        # Both idx and targets have shape (B, T)
        
        # Forward pass through the whole decoder architecture
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C) after broadcast and add
        x = self.blocks(x) # (B, T, C)
        x = self.final_layer_norm(x) # (B, T, C)
        logits = self.lm_head(x)  # (B, T, vocab_size)
        
        # Loss calculation - Cross Entropy (negative log likelihood loss)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # latter block_size part of the sequence
            idx_cond = idx[:, -block_size:]
            # compute the logits and loss
            logits, loss = self(idx_cond)
            # Pick from only the current timestep
            logits = logits[:, -1, :] # (B, T, C) to (B, C)
            probabilites = F.softmax(logits, dim = -1) # (B, C)

            # Sample from the distribution using the probabilites
            idx_next = torch.multinomial(probabilites, num_samples = 1)  # (B, 1)
            idx = torch.cat((idx, idx_next), dim = 1) # Append the sampled token(s) to the running sequence
        
        return idx
            
        
        
    

In [12]:
model = Model.GPTLanguageModel(vocab_size)
m = model.to(device)

# Using AdamW optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iteration in range(max_iters):
    
    # Periodically evaluate loss and once after all training iterations are done
    if iteration % eval_interval == 0 or iteration == max_iters - 1:
        losses = estimate_loss()
        print(f"Step {iteration} : Train Loss  {losses['train']:.4f}, Val Loss {losses['val']:.4f}")
        
    # sample o batch of data
    xb, yb = get_batch('train')
    
    # Evaluate loss
    logits, loss = model(xb, yb)
    print(f"loss : {loss}")
    optimizer.zero_grad(set_to_none = True)  # Setting grads to None rather than 0 for memory efficiency
    loss.backward()
    optimizer.step()

Step 0 : Train Loss  5.0273, Val Loss 5.0222
loss : 5.026833534240723
loss : 3.9979708194732666
loss : 3.7642667293548584
loss : 3.6232588291168213
loss : 3.569612979888916
loss : 3.526869058609009
loss : 3.4389939308166504
loss : 3.439837694168091
loss : 3.349968433380127
loss : 3.3302013874053955
loss : 3.332948684692383
loss : 3.2635531425476074
loss : 3.235618829727173
loss : 3.2352845668792725
loss : 3.182049512863159
loss : 3.2154836654663086
loss : 3.1770501136779785
loss : 3.139188051223755
loss : 3.036560535430908
loss : 3.0504915714263916
loss : 3.0854456424713135
loss : 2.989964723587036
loss : 2.9857208728790283
loss : 2.987921953201294
loss : 2.966843843460083
loss : 2.950360059738159
loss : 2.9682092666625977
loss : 2.917511224746704
loss : 2.854088306427002
loss : 2.9657554626464844
loss : 2.8804538249969482
loss : 2.8189382553100586
loss : 2.8455111980438232
loss : 2.834583044052124
loss : 2.8149781227111816
loss : 2.805117607116699
loss : 2.845028877258301
loss : 2.833

loss : 2.5067803859710693
loss : 2.5174760818481445
loss : 2.467745780944824
loss : 2.4870986938476562
loss : 2.4812729358673096
loss : 2.467996120452881
loss : 2.503274917602539
loss : 2.4929769039154053
loss : 2.4745676517486572
loss : 2.5014805793762207
loss : 2.493717908859253
loss : 2.492433547973633
loss : 2.470937728881836
loss : 2.545311212539673
loss : 2.4894847869873047
loss : 2.4835734367370605
loss : 2.4768354892730713
loss : 2.486135482788086
loss : 2.5146126747131348
loss : 2.4671730995178223
loss : 2.485496759414673
loss : 2.4669301509857178
loss : 2.4848053455352783
loss : 2.514180898666382
loss : 2.4950897693634033
loss : 2.4798362255096436
loss : 2.519735336303711
loss : 2.5284423828125
loss : 2.4899239540100098
loss : 2.4995083808898926
loss : 2.481398582458496
loss : 2.474900245666504
loss : 2.515665292739868
loss : 2.4633235931396484
loss : 2.481372356414795
loss : 2.469581365585327
loss : 2.47287917137146
loss : 2.4733738899230957
loss : 2.4569766521453857
loss : 

loss : 1.7659976482391357
loss : 1.7695481777191162
loss : 1.7502139806747437
loss : 1.7483038902282715
loss : 1.7316685914993286
loss : 1.708818793296814
loss : 1.7161941528320312
loss : 1.7199416160583496
loss : 1.7599539756774902
loss : 1.7769269943237305
loss : 1.731832504272461
loss : 1.6685070991516113
loss : 1.733532190322876
loss : 1.6817659139633179
loss : 1.7376316785812378
loss : 1.7022439241409302
loss : 1.6768996715545654
loss : 1.7318158149719238
loss : 1.7363942861557007
loss : 1.7136496305465698
loss : 1.6791332960128784
loss : 1.7625508308410645
loss : 1.6941344738006592
loss : 1.7104114294052124
loss : 1.691481113433838
loss : 1.7563186883926392
loss : 1.6702812910079956
loss : 1.7108300924301147
loss : 1.7131952047348022
loss : 1.6825960874557495
loss : 1.6845824718475342
loss : 1.6787797212600708
loss : 1.7522671222686768
loss : 1.6807209253311157
loss : 1.7267588376998901
loss : 1.697048306465149
loss : 1.691851019859314
loss : 1.660241723060608
loss : 1.7267003059

loss : 1.4410268068313599
loss : 1.441659688949585
loss : 1.4372210502624512
loss : 1.4663350582122803
loss : 1.4641351699829102
loss : 1.5026602745056152
loss : 1.4820610284805298
loss : 1.4064509868621826
loss : 1.4747952222824097
loss : 1.4445643424987793
loss : 1.4309203624725342
loss : 1.4458106756210327
loss : 1.4695823192596436
loss : 1.459277868270874
loss : 1.4334876537322998
loss : 1.4281340837478638
loss : 1.4375648498535156
loss : 1.4347476959228516
loss : 1.4438318014144897
loss : 1.429190993309021
loss : 1.456862211227417
loss : 1.4449810981750488
loss : 1.4180870056152344
loss : 1.4414576292037964
loss : 1.4388912916183472
loss : 1.4401391744613647
loss : 1.413459062576294
loss : 1.4387439489364624
loss : 1.4895180463790894
loss : 1.4496525526046753
loss : 1.447999119758606
loss : 1.4099879264831543
loss : 1.4377080202102661
loss : 1.4368045330047607
loss : 1.429983139038086
loss : 1.374229073524475
loss : 1.4373712539672852
loss : 1.450553059577942
loss : 1.412372350692

KeyboardInterrupt: 

In [13]:
torch.save(m.state_dict(), './models/textbook-trained-model')

In [14]:
model = Model.GPTLanguageModel(vocab_size)
model = model.to(device)
model.load_state_dict(torch.load('./models/textbook-trained-model'))

<All keys matched successfully>

In [15]:
model.eval()

GPTLanguageModel(
  (token_embedding_table): Embedding(144, 384)
  (position_embedding_table): Embedding(256, 384)
  (blocks): Sequential(
    (0): Block(
      (self_attention): MultiHeadedAttention(
        (heads): ModuleList(
          (0): Head(
            (query): Linear(in_features=384, out_features=64, bias=False)
            (key): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
          (1): Head(
            (query): Linear(in_features=384, out_features=64, bias=False)
            (key): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
          (2): Head(
            (query): Linear(in_features=384, out_features=64, bias=False)
            (key): Linear(in_features=384, out_features=64, bia

In [22]:
def generate_more(input_text):
    encoded_context = torch.as_tensor(encode(input_text), dtype = torch.long, device = device)
    encoded_context = torch.stack((encoded_context,)) # To turn it into a tensor of dimensions (1, context_length)
    n = len(input_text)
    encoded_text = model.generate(encoded_context, max_new_tokens = random.randint(20, 200))[0].tolist()
    decoded_response = decode(encoded_text)[n:]
    return decoded_response, len(decoded_response) 

In [32]:
# Far from generating relevant stuff (nothing unexpected)
response, size = generate_more('Processes')
print(f'Generation ({size} characters) : {response}')

Generation (106 characters) : 
ared implementations functions and evices of subsystem fthe can be extender for
and eading to access syst
